## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

http://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

In [1]:
import pandas as pd

bookings_file = 'bookings.csv.bz2'
searches_file = 'searches.csv.bz2'

In [2]:
bookings_sample = pd.read_csv(bookings_file, sep = '^', nrows = 100000)

In [3]:
pd.options.display.max_columns = None

bookings_sample.head(2)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL


In [ ]:
# bookings.drop_duplicates()

Esto va a tardar un huevo porque para quitar duplicados tiene que ordenar. Ordenar es de lo más caro computacionalmente. Además de tardar mucho puede petar.

Otra manera de hacerlo sería con chunks. Sería así, pero no lo ejecutamos porque tarda mucho

In [ ]:
bookings = pd.read_csv(bookings_file, sep = '^', chunksize = 1000000)
partial_results = []

for chunk in bookings:
    partial_results.append(chunk.drop_duplicates())
    
bookings_nodups = pd.concat(partial_results).drop_duplicates()

Esto tiene un problema ya que si los duplicados están en distintos chunks no me los va a quitar. Al final, con el consolidado habría que volver a hacer un `drop_duplicates()`

#### We have seen that we have white space in some columns....

In [4]:
bookings_sample.columns = bookings_sample.columns.str.strip()

###  Searches

In [5]:
searches_sample = pd.read_csv(searches_file, sep = '^', nrows = 100000)
searches_sample.head(2)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV


## 3) Make action plan

Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

1) remove duplicates

2) parse dates from string to datetime

3) remove whitespaces

        a) from colum names
        
        b) from content
        
4) remove NaN

5) define the model

    if there is one booking for a given O&D done at the same day as the search (for a given O&D), ALL searches of the day (for a given 0&D) might have resulted from the same source and will be set with 1.
    This is regardless of the boarding time of the plane... So if I was looking for plane for the first 4 days of December for a given O&D all those searches would be set to 1 not just the one correspoding to the correct boarding time

        match
        Search : [search_date, O&D] 
        Booking: [cre_date, O&D]

6) execute the model

        a) Group by bookings on [Activity_date, O&D] so that we dont have duplicates 
        (and we can have number of bookings for the day)
        or we can just drop the duplicates
        b) search left join bookings adding "Booked" column
        c) test if the merge was done right
        d) fill NaN of "booked" column with 0
        e) pull all values of booked column >1 to 1

more complex... get number of segments from searches
match search_date, and then split all O&D of all segments, and match the date of first flight of each segment (seg1Date, seg2Date)
with boarding time and O&D and act_date of booking

What do we have?

In [6]:
bookings_sample.head(2)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL


In [7]:
non_cancelled = bookings_sample[bookings_sample['pax'] > 0]  # Quitamos las cancelaciones

non_cancelled = non_cancelled[['cre_date', 'dep_port', 'arr_port']]

In [8]:
non_cancelled.head(2)  # Faltan las fechas

,cre_date,dep_port,arr_port
1,2013-03-26 00:00:00,SAL,CLT
2,2013-03-26 00:00:00,SAL,CLT


In [9]:
non_cancelled['cre_date'] = non_cancelled['cre_date'].str[:10]

In [10]:
non_cancelled.head(2)

,cre_date,dep_port,arr_port
1,2013-03-26,SAL,CLT
2,2013-03-26,SAL,CLT


In [11]:
non_cancelled.shape

(68654, 3)

Ahora hay que quitar duplicados sí o sí para el join

In [12]:
non_cancelled = non_cancelled.drop_duplicates()

Trabajamos la tabla entera

In [13]:
%%time

from datetime import datetime

bookings = pd.read_csv(bookings_file, sep = '^', chunksize = 1000000, usecols = ['cre_date           ', 'dep_port', 'arr_port', 'pax'])

partial_results = []

for chunk_num, chunk in enumerate(bookings):
    
    print("Start with chunk % d in bookings at %s" % (chunk_num, datetime.now()))
    
    chunk.columns = chunk.columns.str.strip()
    non_cancelled = chunk[chunk['pax'] > 0]  # Quitamos las cancelaciones
    non_cancelled['cre_date'] = non_cancelled['cre_date'].str[:10]
    non_cancelled = non_cancelled[['cre_date', 'dep_port', 'arr_port']]
    
    partial_results.append(non_cancelled.drop_duplicates())
    
bookings_nodups = pd.concat(partial_results).drop_duplicates()

Start with chunk  0 in bookings at 2018-06-02 09:08:48.417813


/home/dsc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Start with chunk  1 in bookings at 2018-06-02 09:09:06.282287
Start with chunk  2 in bookings at 2018-06-02 09:09:23.794081
Start with chunk  3 in bookings at 2018-06-02 09:09:41.313556
Start with chunk  4 in bookings at 2018-06-02 09:09:58.910231
Start with chunk  5 in bookings at 2018-06-02 09:10:16.532661
Start with chunk  6 in bookings at 2018-06-02 09:10:34.656941
Start with chunk  7 in bookings at 2018-06-02 09:10:52.939835
Start with chunk  8 in bookings at 2018-06-02 09:11:10.984976
Start with chunk  9 in bookings at 2018-06-02 09:11:29.551189
Start with chunk  10 in bookings at 2018-06-02 09:11:30.072175
CPU times: user 2min 52s, sys: 4.6 s, total: 2min 56s
Wall time: 3min


In [14]:
bookings_nodups.head()

,cre_date,dep_port,arr_port
1,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
5,2013-03-20,DEN,LGA
7,2013-03-25,NRT,SIN
19,2013-03-13,YYZ,TUS


In [15]:
bookings_nodups.shape

(315574, 3)

In [16]:
searches_sample.head(2)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV


Como searches es muy grande pero bookings es muy pequeñito, podemos hacer el join por chunks. Si los dos fueran grandes, hacer un join por chunk no se puede.

In [17]:
merged = searches_sample.merge(bookings_nodups, 
                               left_on = ['Origin', 'Destination', 'Date'], 
                               right_on = ['dep_port', 'arr_port', 'cre_date'])

In [18]:
merged  # Mala pinta

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port


In [19]:
searches_sample['Origin'].str.len().mean()

3.0

In [20]:
bookings_nodups['dep_port'].str.len().mean()

8.0

No tienen los mismos caracteres. Por eso no cruza.

In [21]:
%%time

from datetime import datetime

bookings = pd.read_csv(bookings_file,
                       sep = '^', 
                       chunksize = 1000000, 
                       usecols = ['cre_date           ', 'dep_port', 'arr_port', 'pax'])

partial_results = []

for chunk_num, chunk in enumerate(bookings):
    
    print("Start with chunk % d in bookings at %s" % (chunk_num, datetime.now()))
    
    chunk.columns = chunk.columns.str.strip()
    chunk['dep_port'] = chunk['dep_port'].str.strip()
    chunk['arr_port'] = chunk['arr_port'].str.strip()
    
    non_cancelled = chunk[chunk['pax'] > 0]  # Quitamos las cancelaciones
    non_cancelled['cre_date'] = non_cancelled['cre_date'].str[:10]
    non_cancelled = non_cancelled[['cre_date', 'dep_port', 'arr_port']]
    
    partial_results.append(non_cancelled.drop_duplicates())
    
bookings_nodups = pd.concat(partial_results).drop_duplicates()

Start with chunk  0 in bookings at 2018-06-02 09:12:21.499806


/home/dsc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Start with chunk  1 in bookings at 2018-06-02 09:12:39.889393
Start with chunk  2 in bookings at 2018-06-02 09:12:58.164027
Start with chunk  3 in bookings at 2018-06-02 09:13:16.362436
Start with chunk  4 in bookings at 2018-06-02 09:13:34.976959
Start with chunk  5 in bookings at 2018-06-02 09:13:53.867866
Start with chunk  6 in bookings at 2018-06-02 09:14:13.029956
Start with chunk  7 in bookings at 2018-06-02 09:14:31.389252
Start with chunk  8 in bookings at 2018-06-02 09:14:51.289492
Start with chunk  9 in bookings at 2018-06-02 09:15:09.583660
Start with chunk  10 in bookings at 2018-06-02 09:15:10.925678
CPU times: user 3min 6s, sys: 1.26 s, total: 3min 7s
Wall time: 3min 8s


In [22]:
merged = searches_sample.merge(bookings_nodups, 
                               left_on = ['Origin', 'Destination', 'Date'], 
                               right_on = ['dep_port', 'arr_port', 'cre_date'],
                               how = 'left')

In [23]:
merged.head(2)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,NaN,NaN,NaN
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,NaN,NaN,NaN


In [24]:
searches_sample.shape

(100000, 45)

In [25]:
merged.shape  # OK

(100000, 48)

In [26]:
# Los que no han cruzado tendrán en alguna de las tres NaN

merged['Booking'] = merged['cre_date'].notnull().astype(int)

In [27]:
merged.head(2)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port,Booking
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,NaN,NaN,NaN,0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,NaN,NaN,NaN,0


In [28]:
merged = merged.drop(['cre_date', 'dep_port', 'arr_port'], axis = 1)

In [29]:
merged.head(2)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Booking
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0


Ahora extendámoslo a chunks

In [ ]:
%%time

searches = pd.read_csv(searches_file, sep = '^', chunksize = 2e6)

partial_results

for chunk_num, chunk in enumerate(searches):
    
    print("Start with chunk % d in bookings at %s" % (chunk_num, datetime.now()))
 
    merged = chunk.merge(bookings_nodups, 
                               left_on = ['Origin', 'Destination', 'Date'], 
                               right_on = ['dep_port', 'arr_port', 'cre_date'],
                               how = 'left')
    
    merged['Booking'] = merged['cre_date'].notnull().astype(int)
    merged = merged.drop(['cre_date', 'dep_port', 'arr_port'], axis = 1)
    
  # Vamos añadiendo el resultado de cada chunk al .csv  

    merged.to_csv('annotated_searches.csv',
                  compression='bz2', 
                  sep='^', 
                  mode='a',
                  header= not bool(chunk_num))  # Esto es para que sólo me escriba el header en la primera iteración.
    